In [ ]:
import datetime
import dateutil
import time
import uuid
import random
import json

In [ ]:
a = 1
b = 10
data = []
date = datetime.date(2021, 1, 1)

In [ ]:
for a in range(b):
    data.append(
        {
            'id': str(uuid.uuid4()),
            'code': a,
            'option': 'option {0}'.format(random.randint(1,5)),
            'description': 'test{0}'.format(a),
            'value': round(random.random() * 1000, 2),
            'rate': round(random.random(), 4),
            'start_date': (date - datetime.date(1970,1,1)).days,
            'created_at': int((datetime.datetime.today() - datetime.datetime(1970,1,1)).total_seconds() * 1000000),
            'updated_at': int((datetime.datetime.today() - datetime.datetime(1970,1,1)).total_seconds() * 1000000),
            'status': True if random.random() < 0.5 else False,
            'detail': {
                'detail_id': a + 1000,
                'description': 'description{0}'.format(a)
            }
        }
    )

In [1]:
import uuid
from pyspark.sql import SparkSession

In [2]:
sparkSession = SparkSession.builder \
    .config('spark.jars.packages', 'org.apache.spark:spark-avro_2.12:3.0.0') \
    .getOrCreate()

21/08/17 23:42:56 WARN Utils: Your hostname, ubuntu-vm-01 resolves to a loopback address: 127.0.1.1; using 192.168.15.130 instead (on interface enp0s3)
21/08/17 23:42:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/danilo/Documents/env/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/danilo/.ivy2/cache
The jars for the packages stored in: /home/danilo/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b809b137-7fd1-466e-ad92-13551e095ba8;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.0.0 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution report :: resolve 213ms :: artifacts dl 9ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.0.0 from central in [default]
	org.spark-project.spark#unused;1.0.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	--------------------------------

In [ ]:
from pyspark.sql.types import *
schema = StructType([
    StructField('id', StringType(), True),
    StructField('code', LongType(), True),
    StructField('option', StringType(), True),
    StructField('description', StringType(), True),
    StructField('value', DoubleType(), True),
    StructField('rate', DoubleType(), True),
    StructField('start_date', DateType(), True),
    StructField('created_at', TimestampType(), True),
    StructField('updated_at', TimestampType(), True),
    StructField('status', StringType(), True),
    StructField('detail', MapType(LongType(), StringType(), True), True)
])

In [ ]:
df = sparkSession.createDataFrame(data)

In [ ]:
df.printSchema()

In [ ]:
df.write.format("avro").save("hdfs://dataserver:9000/warehouse/{0}".format(uuid.uuid4()))

In [3]:
df = sparkSession.read.format("avro").load('/home/danilo/Documents/batch01/fact01/2010/1/data_file_20100101.avro')

In [6]:
df.show()

+--------------------+----+--------+-----------+------+------+----------+----------------+----------------+------+--------------------+
|                  id|code|  option|description| value|  rate|start_date|      created_at|      updated_at|status|              detail|
+--------------------+----+--------+-----------+------+------+----------+----------------+----------------+------+--------------------+
|c5a3d08a-24a2-407...|   1|option 1|      test1| 357.8|0.9408|2010-01-01|1629243587060357|1629243587060367|  true|{1001, description1}|
|bc74faf2-722d-438...|   2|option 2|      test2|374.81| 0.791|2010-01-01|1629243587060589|1629243587060595| false|{1002, description2}|
|9d3b1b55-85c6-490...|   3|option 3|      test3|762.49|0.6876|2010-01-01|1629243587060734|1629243587060738|  true|{1003, description3}|
|b6feaf47-cce3-437...|   4|option 4|      test4|962.53|0.8617|2010-01-01|1629243587060898|1629243587060901| false|{1004, description4}|
|56ac675e-7ea5-4f5...|   5|option 2|      test5|